In [1]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
warnings.filterwarnings("ignore")

In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM, AdamW, AutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split
import numpy as np

In [3]:

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from sklearn.metrics import accuracy_score

In [4]:
df_enc = pd.read_csv('enciklopediyastroy.csv')
df_enc.columns = ['record_name', 'ref_name']
df_enc['labels'] = 1
df_enc

,record_name,ref_name,labels
0,Поднятие волокон,"Набухание и поднятие коротких, поврежденных во...",1
1,Несовместимость,"Имеет место, когда один лакокрасочный продукт ...",1
2,Твердость поверхностная,Способность лакокрасочной пленки выдерживать м...,1
3,Пистолет-краскораспылитель,Устройство для нанесения красочного покрытия «...,1
4,Очистка стружкой,"Процедура, приводящая к устранению шероховатос...",1
...,...,...,...
32587,RUBENIS,Водный раствор поликарбоксилатного эфира опред...,1
32588,Sika ® ViscoCrete ® -10 R,Суперпластификатор для бетонов и растворов. пр...,1
32589,Sika ® ViscoCrete ® -3210,"Суперпластификатор, используется для сборного ...",1
32590,Sika® FM 95,Суперпластификатор для получения бетонов с выс...,1


In [5]:
# # Пример данных
# terms = df_enc['record_name'].tolist()
# definitions = df_enc['ref_name'].tolist()

# # Объединение терминов и значений для предобучения
# texts = [f"{term} : {definition}" for term, definition in zip(terms, definitions)]

# # Разделение данных на тренировочные и тестовые наборы
# train_texts, val_texts = train_test_split(texts, test_size=0.2)

# # Загрузка токенизатора и модели
# tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-MiniLM-L6-v2')
# model = AutoModelForMaskedLM.from_pretrained('sentence-transformers/paraphrase-MiniLM-L6-v2')

# # Настройка устройства
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)

# # Создание датасета
# class TextDataset(Dataset):
#     def __init__(self, texts, tokenizer, max_len=512):
#         self.texts = texts
#         self.tokenizer = tokenizer
#         self.max_len = max_len

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, idx):
#         text = self.texts[idx]

#         encoding = self.tokenizer(
#             text,
#             max_length=self.max_len,
#             padding='max_length',
#             truncation=True,
#             return_tensors='pt'
#         )

#         input_ids = encoding['input_ids'].flatten()
#         attention_mask = encoding['attention_mask'].flatten()

#         # Создание меток для задачи MLM
#         labels = input_ids.clone()
#         probability_matrix = torch.full(labels.shape, 0.15)
#         masked_indices = torch.bernoulli(probability_matrix).bool()
#         labels[~masked_indices] = -100  # Мы не вычисляем лосс для этих токенов

#         return {
#             'input_ids': input_ids,
#             'attention_mask': attention_mask,
#             'labels': labels
#         }

# train_dataset = TextDataset(train_texts, tokenizer)
# val_dataset = TextDataset(val_texts, tokenizer)

# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=16)

# # Определение оптимизатора и лосса
# optimizer = AdamW(model.parameters(), lr=5e-5)
# loss_fn = torch.nn.CrossEntropyLoss()
# epochs = 20
# # Процесс обучения
# for epoch in tqdm(range(epochs)):
#     model.train()
#     train_loss = 0
#     for batch in tqdm(train_loader):
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         train_loss += loss.item()
#         loss.backward()

#         optimizer.step()
#         optimizer.zero_grad()

#     avg_train_loss = train_loss / len(train_loader)
#     print(f"Average train loss for epoch {epoch}: {avg_train_loss}")

#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)

#             outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#             loss = outputs.loss
#             val_loss += loss.item()

#     avg_val_loss = val_loss / len(val_loader)
#     print(f"Average validation loss for epoch {epoch}: {avg_val_loss}")


In [6]:
# # Сохранение модели после предобучения
# model.save_pretrained('pretrained_model')
# tokenizer.save_pretrained('pretrained_model')


In [7]:
#КСР качаем от сюда https://minstroyrf.gov.ru/trades/tsenoobrazovanie/klassifikator-stroitelnykh-resursov/
#Читаем КСР
df = pd.read_excel('Классификатор 15022024.xlsx',header=1, engine='openpyxl')
df = df.dropna()
for i in ['Книга','Часть','Раздел','Группа']:
    df[i] = df['Код ресурса'].map(lambda x: x.split(':')[0] if i in x else np.nan)
    df[i] = df[i].fillna(method='ffill')
df = df.dropna()
df = df[~(df['Код ресурса'] == df['Наименование'])]
df = df[["Книга","Часть","Раздел","Группа","Код ресурса","Наименование","Ед.изм."]]
df_m = df
df = pd.read_excel('Классификатор 15022024.xlsx',header=1, engine='openpyxl', sheet_name=1)
df = df.dropna()
for i in ['Книга','Часть','Раздел','Группа']:
    df[i] = df['Код ресурса'].map(lambda x: x.split(':')[0] if i in x else np.nan)
    df[i] = df[i].fillna(method='ffill')

df = df[~(df['Код ресурса'] == df['Наименование'])]
df = df[["Книга","Часть","Раздел","Группа","Код ресурса","Наименование","Ед.изм."]]
df = pd.concat([df_m, df])
df

,Книга,Часть,Раздел,Группа,Код ресурса,Наименование,Ед.изм.
4,Книга 01,Часть 01.1,Раздел 01.1.01,Группа 01.1.01.01,01.1.01.01-0002,Детали фасонные коньковые к листам хризотилцем...,100 компл
5,Книга 01,Часть 01.1,Раздел 01.1.01,Группа 01.1.01.01,23.65.12.190.01.1.01.01-0002-000,Детали фасонные коньковые к листам хризотилцем...,100 компл
7,Книга 01,Часть 01.1,Раздел 01.1.01,Группа 01.1.01.02,01.1.01.02-0011,"Доска электротехническая дугостойкая (АЦЭИД), ...",т
8,Книга 01,Часть 01.1,Раздел 01.1.01,Группа 01.1.01.02,23.65.12.190.01.1.01.02-0011-000,"Доска электротехническая дугостойкая (АЦЭИД), ...",т
10,Книга 01,Часть 01.1,Раздел 01.1.01,Группа 01.1.01.04,01.1.01.04-1018,"Листы хризотилцементные волнистые, профиль 40/...",м2
...,...,...,...,...,...,...,...
3592,Книга 96,NaN,Раздел 96.01,Группа 96.01.15,28.92.50.000.96.01.15-002-000,"Тракторы на гусеничном ходу, мощность 59 кВт (...",шт
3594,Книга 96,NaN,Раздел 96.01,Группа 96.01.17,96.01.17-001,Агрегаты для сварки полиэтиленовых труб,маш.-ч
3595,Книга 96,NaN,Раздел 96.01,Группа 96.01.17,27.90.31.110.96.01.17-001-000,Агрегаты для сварки полиэтиленовых труб,шт
3596,Книга 96,NaN,Раздел 96.01,Группа 96.01.17,96.01.17-002,Агрегаты сварочные передвижные с дизельным дви...,маш.-ч


In [8]:
df_train = pd.read_excel('/home/dimk/langchain/KSR/train.xlsx', engine='openpyxl')
df_train

,record_name,record_name_2,ref_code,ref_name,ref_unit
0,DIN -рейка оцинкованная ТН35-7.5 100 см (Chint),"DIN-рейка 35х7,5 мм длиной 1000 мм",27.33.13.130.20.2.08.01-0001-000,"DIN-рейки металлические, оцинкованные, размеры...",100 шт
1,Анкерный элемент ТехноНиколь 8*45мм,Анкерный элемент TN 8x4.5,25.94.11.190.01.7.15.01-1169-000,"Анкер грунтовый забивной самораскрывающийся, о...",шт
2,Анкер забивной М10 DRM 12x40 сталь,Анкер втулочный M10,25.94.11.190.01.7.15.01-0037-000,"Анкер забивной латунный, диаметр внутренней ре...",шт
3,Анкер забивной М10/12x40,Анкер втулочный M10,25.94.11.190.01.7.15.01-0037-000,"Анкер забивной латунный, диаметр внутренней ре...",шт
4,Анкер забивной М8 LAZ латунь,Анкер втулочный M8,25.94.11.190.01.7.15.01-0036-000,"Анкер забивной латунный, диаметр внутренней ре...",шт
...,...,...,...,...,...
2402,Корпус замка Фабрика замков L 008,Межкомнатные двери 900х2000,27.12.31.000.20.4.04.04-0002-000,Щит учетно-распределительный встраиваемый ЩУРВ...,шт
2403,Эмаль Кремнийорганическая Термостойкая (8) 81....,Кремнийорганическая эмаль КО-8104,20.30.12.130.14.4.04.04-0010-005,Эмаль кремнийорганическая термостойкая КО-811К...,т
2404,Кремнийорганическая эмаль КО-8104,Кремнийорганическая эмаль КО-8104,20.30.12.130.14.4.04.04-0001-000,Эмаль кремнийорганическая термостойкая КО-88,т
2405,Кремнийорганическая эмаль КО-8104 (серебристо ...,Кремнийорганическая эмаль КО-8104,20.30.12.130.14.4.04.04-0001-000,Эмаль кремнийорганическая термостойкая КО-88,т


In [9]:
train = df_train[['record_name', 'ref_name']]
train['labels'] = 0

In [10]:
train['labels'] = (train['record_name'] == train['ref_name'])
train['labels'] = train['labels'].astype(int)

In [11]:
train['labels'] = train['labels'].map(lambda x: 0.9 if x == 0 else x)

In [12]:
train_1 = df_train[(df_train['record_name'] != df_train['record_name_2'])][['record_name_2', 'ref_name']]
train_1['labels'] = (train_1['record_name_2'] == train_1['ref_name'])
train_1['labels'] = train_1['labels'].map(lambda x: 0.7 if x == 0 else x)

In [13]:
train_1.columns = ['record_name', 'ref_name', 'labels']
train_1

,record_name,ref_name,labels
0,"DIN-рейка 35х7,5 мм длиной 1000 мм","DIN-рейки металлические, оцинкованные, размеры...",0.7
1,Анкерный элемент TN 8x4.5,"Анкер грунтовый забивной самораскрывающийся, о...",0.7
2,Анкер втулочный M10,"Анкер забивной латунный, диаметр внутренней ре...",0.7
3,Анкер втулочный M10,"Анкер забивной латунный, диаметр внутренней ре...",0.7
4,Анкер втулочный M8,"Анкер забивной латунный, диаметр внутренней ре...",0.7
...,...,...,...
2400,Щит стеклопластиковый серии СП400 размерами 10...,Щит распределительный наружной установки ЩРН-1...,0.7
2401,Щиток ПВХ бокс на 6 мест,Щит распределительный наружной установки ЩРН-6...,0.7
2402,Межкомнатные двери 900х2000,Щит учетно-распределительный встраиваемый ЩУРВ...,0.7
2403,Кремнийорганическая эмаль КО-8104,Эмаль кремнийорганическая термостойкая КО-811К...,0.7


In [14]:
train_2 = df_train[['record_name', 'record_name_2']]
train_2['labels'] = (train_2['record_name'] == train_2['record_name_2'])
train_2['labels'] = train_2['labels'].astype(int)
train_2['labels'] = train_2['labels'].map(lambda x: 0.8 if x == 0 else x)

In [15]:
train_3 = df[['Наименование']]
train_3['ref_name'] = np.random.choice(df_enc['record_name'].tolist(), size=len(train_3))
train_3['labels'] = 0
train_3.columns = ['record_name', 'ref_name', 'labels']
train_3

,record_name,ref_name,labels
4,Детали фасонные коньковые к листам хризотилцем...,Ресурс коммунальный,0
5,Детали фасонные коньковые к листам хризотилцем...,Стекло тонированное напыление,0
7,"Доска электротехническая дугостойкая (АЦЭИД), ...",Координаты цветовые,0
8,"Доска электротехническая дугостойкая (АЦЭИД), ...",Сталь,0
10,"Листы хризотилцементные волнистые, профиль 40/...","Требования к клеям, экологические",0
...,...,...,...
3592,"Тракторы на гусеничном ходу, мощность 59 кВт (...",Размер выступа,0
3594,Агрегаты для сварки полиэтиленовых труб,Упаковка готовая,0
3595,Агрегаты для сварки полиэтиленовых труб,Гидрошпонка,0
3596,Агрегаты сварочные передвижные с дизельным дви...,Риск поставщика,0


In [16]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import random
from razdel import tokenize
import pymorphy2
from ruwordnet import RuWordNet


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/dimk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/dimk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/dimk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [17]:
def random_insertion(sentence, n=1):
    words = nltk.word_tokenize(sentence)
    for _ in range(n):
        idx = random.randint(0, len(words)-1)
        words.insert(idx, random.choice(words))
    return ' '.join(words)

def random_deletion(sentence, p=0.2):
    words = nltk.word_tokenize(sentence)
    remaining_words = [word for word in words if random.uniform(0, 1) > p]
    return ' '.join(remaining_words)

def word_order_shuffle(sentence):
    words = nltk.word_tokenize(sentence)
    random.shuffle(words)
    return ' '.join(words)

def random_char_modification(word):
    if random.choice([True, False]):
        # Insert a random character
        char = random.choice('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZабвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ')
        pos = random.randint(0, len(word))
        word = word[:pos] + char + word[pos:]
    else:
        # Delete a random character
        if len(word) > 1:
            pos = random.randint(0, len(word) - 1)
            word = word[:pos] + word[pos+1:]
    return word

def random_word_modification(sentence, percentage=0.7):
    words = nltk.word_tokenize(sentence)
    num_words_to_modify = int(len(words) * percentage)
    indices_to_modify = random.sample(range(len(words)), num_words_to_modify)
    
    for idx in indices_to_modify:
        modification_type = random.choice(['shuffle', 'insert', 'delete', 'char_mod'])
        if modification_type == 'shuffle':
            words[idx] = word_order_shuffle(words[idx])
        elif modification_type == 'insert':
            words[idx] = random_insertion(words[idx])
        elif modification_type == 'delete':
            words[idx] = random_deletion(words[idx])
        elif modification_type == 'char_mod':
            words[idx] = random_char_modification(words[idx])
    
    return ' '.join(words)

# Пример использования функций аугментации
sentence = "Кошка спит одна на подоконнике."
print("Исходное предложение:", sentence)
print("Вставка случайных слов или фраз:", random_insertion(sentence))
print("Удаление слов или фраз:", random_deletion(sentence))
print("Перестановка порядка слов:", word_order_shuffle(sentence))
print("Случайные изменения слов (70% слов):", random_word_modification(sentence, 0.7))



Исходное предложение: Кошка спит одна на подоконнике.
Вставка случайных слов или фраз: подоконнике Кошка спит одна на подоконнике .
Удаление слов или фраз: Кошка спит одна подоконнике .
Перестановка порядка слов: подоконнике на Кошка спит одна .
Случайные изменения слов (70% слов):  спит одна на подоконнике .


In [18]:
aug = [random_deletion,word_order_shuffle,random_word_modification]
res = []
for i in tqdm(aug):
     train_4 = df[['Наименование']]
     train_4['ref_name'] = train_4['Наименование'].map(i)
     res.append(train_4)
     train_4['labels'] = 0.85
     train_4.columns = ['record_name', 'ref_name', 'labels']
train_4 = pd.concat(res)
train_4 

  0%|          | 0/3 [00:00<?, ?it/s]

,record_name,ref_name,labels
4,Детали фасонные коньковые к листам хризотилцем...,Детали фасонные коньковые к листам хризотилцем...,0.85
5,Детали фасонные коньковые к листам хризотилцем...,Детали коньковые к листам хризотилцементным во...,0.85
7,"Доска электротехническая дугостойкая (АЦЭИД), ...",Доска электротехническая ( АЦЭИД ) толщина 6,0.85
8,"Доска электротехническая дугостойкая (АЦЭИД), ...",Доска электротехническая дугостойкая ( АЦЭИД )...,0.85
10,"Листы хризотилцементные волнистые, профиль 40/...","хризотилцементные волнистые , профиль , 7-волн...",0.85
...,...,...,...
3592,"Тракторы на гусеничном ходу, мощность 59 кВт (...","Тракторы на гусеничном хоЕду , мощность 9 кВт ...",0.85
3594,Агрегаты для сварки полиэтиленовых труб,Агрегаты для сварки полиэтиленовых труб,0.85
3595,Агрегаты для сварки полиэтиленовых труб,Агрегаты для свари полиэтиленовых труб,0.85
3596,Агрегаты сварочные передвижные с дизельным дви...,Агрегаты сварочные передвижные с с дизельным д...,0.85


In [19]:
train_5 = df[['Наименование']]
train_5['ref_name'] = df['Наименование']
train_5['labels'] = 1
train_5.columns = ['record_name', 'ref_name', 'labels']
train_5

,record_name,ref_name,labels
4,Детали фасонные коньковые к листам хризотилцем...,Детали фасонные коньковые к листам хризотилцем...,1
5,Детали фасонные коньковые к листам хризотилцем...,Детали фасонные коньковые к листам хризотилцем...,1
7,"Доска электротехническая дугостойкая (АЦЭИД), ...","Доска электротехническая дугостойкая (АЦЭИД), ...",1
8,"Доска электротехническая дугостойкая (АЦЭИД), ...","Доска электротехническая дугостойкая (АЦЭИД), ...",1
10,"Листы хризотилцементные волнистые, профиль 40/...","Листы хризотилцементные волнистые, профиль 40/...",1
...,...,...,...
3592,"Тракторы на гусеничном ходу, мощность 59 кВт (...","Тракторы на гусеничном ходу, мощность 59 кВт (...",1
3594,Агрегаты для сварки полиэтиленовых труб,Агрегаты для сварки полиэтиленовых труб,1
3595,Агрегаты для сварки полиэтиленовых труб,Агрегаты для сварки полиэтиленовых труб,1
3596,Агрегаты сварочные передвижные с дизельным дви...,Агрегаты сварочные передвижные с дизельным дви...,1


In [20]:
def random_deletion(sentence, p=0.7):
    words = nltk.word_tokenize(sentence)
    remaining_words = [word for word in words if random.uniform(0, 1) > p]
    return ' '.join(remaining_words)
    
train_6 = df[['Наименование']]
train_6['ref_name'] = df['Наименование'].map(random_insertion)
train_6['ref_name'] = train_6['ref_name'].map(random_deletion)
train_6['labels'] = 0.4
train_6.columns = ['record_name', 'ref_name', 'labels']
train_6

,record_name,ref_name,labels
4,Детали фасонные коньковые к листам хризотилцем...,коньковые хризотилцементным волнистым,0.4
5,Детали фасонные коньковые к листам хризотилцем...,,0.4
7,"Доска электротехническая дугостойкая (АЦЭИД), ...","Доска дугостойкая ,",0.4
8,"Доска электротехническая дугостойкая (АЦЭИД), ...",дугостойкая 6,0.4
10,"Листы хризотилцементные волнистые, профиль 40/...",хризотилцементные толщина,0.4
...,...,...,...
3592,"Тракторы на гусеничном ходу, мощность 59 кВт (...","Тракторы , ) мощность л.с",0.4
3594,Агрегаты для сварки полиэтиленовых труб,Агрегаты для сварки,0.4
3595,Агрегаты для сварки полиэтиленовых труб,сварки,0.4
3596,Агрегаты сварочные передвижные с дизельным дви...,Агрегаты сварочные с двигателем сварочный,0.4


In [21]:
def random_deletion(sentence, p=0.7):
    words = nltk.word_tokenize(sentence)
    remaining_words = [word for word in words if random.uniform(0, 1) > p]
    return ' '.join(remaining_words)
    
train_7 = df[['Наименование']]
train_7['ref_name'] = df['Наименование'].map(random_insertion)
train_7['ref_name'] = train_7['ref_name'].map(random_deletion)
train_7['ref_name'] = train_7['ref_name'].map(random_word_modification)
train_7['labels'] = 0.3
train_7.columns = ['record_name', 'ref_name', 'labels']
train_7

,record_name,ref_name,labels
4,Детали фасонные коньковые к листам хризотилцем...,листам хризотилцементным,0.3
5,Детали фасонные коньковые к листам хризотилцем...,листам,0.3
7,"Доска электротехническая дугостойкая (АЦЭИД), ...",мм,0.3
8,"Доска электротехническая дугостойкая (АЦЭИД), ...",Доска электротехническая Доска АЦЭИД толщина,0.3
10,"Листы хризотилцементные волнистые, профиль 40/...","волистые , , , толщина 52",0.3
...,...,...,...
3592,"Тракторы на гусеничном ходу, мощность 59 кВт (...",Тракторы мсощность (,0.3
3594,Агрегаты для сварки полиэтиленовых труб,для,0.3
3595,Агрегаты для сварки полиэтиленовых труб,труб,0.3
3596,Агрегаты сварочные передвижные с дизельным дви...,"Агрегаты Агрегаты А , сварочный ток",0.3


In [22]:
# train_8 = df[['Наименование']]
# train_8['ref_name'] = np.random.choice(df_enc['ref_name'].tolist(), size=len(train_8))
# train_8['labels'] = 0
# train_8.columns = ['record_name', 'ref_name', 'labels']
# train_8

In [23]:
test = train
train = pd.concat([train_1,train_3, train_4, train_5, train_6, train_7])
len(train), len(test)

(1493804, 2407)

In [24]:
# Разделение данных на тренировочные и тестовые наборы
train_pairs = list(train.itertuples(index=False, name=None))
val_pairs = list(test.itertuples(index=False, name=None))

In [25]:
val_pairs[0]

('DIN -рейка оцинкованная ТН35-7.5 100 см (Chint)',
 'DIN-рейки металлические, оцинкованные, размеры 7,5х35х260 мм',
 0.9)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM, AdamW
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast

# Загрузка токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained('pretrained_model')
model = AutoModel.from_pretrained('pretrained_model')

# Настройка устройства
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

class CosineSimilarityDataset(Dataset):
    def __init__(self, pairs, tokenizer, max_len=256):
        self.pairs = pairs
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        sent1, sent2, label = self.pairs[idx]

        encoding1 = self.tokenizer(
            sent1,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        encoding2 = self.tokenizer(
            sent2,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids1': encoding1['input_ids'].flatten(),
            'attention_mask1': encoding1['attention_mask'].flatten(),
            'input_ids2': encoding2['input_ids'].flatten(),
            'attention_mask2': encoding2['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.float)
        }

train_dataset = CosineSimilarityDataset(train_pairs, tokenizer)
val_dataset = CosineSimilarityDataset(val_pairs, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)


class CosineSimilarityModel(nn.Module, AutoModel):
    def __init__(self, pretrained_model):
        super(CosineSimilarityModel, self).__init__()
        self.pretrained_model = pretrained_model
        self.pooling = nn.AdaptiveAvgPool1d(1)
        self.cosine_similarity = nn.CosineSimilarity(dim=1)

    def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2):
        outputs1 = self.pretrained_model(input_ids1, attention_mask=attention_mask1)
        outputs2 = self.pretrained_model(input_ids2, attention_mask=attention_mask2)
        
        pooled_output1 = self.pooling(outputs1.last_hidden_state.transpose(1, 2)).squeeze(-1)
        pooled_output2 = self.pooling(outputs2.last_hidden_state.transpose(1, 2)).squeeze(-1)
        
        cosine_sim = self.cosine_similarity(pooled_output1, pooled_output2)
        return cosine_sim

model = CosineSimilarityModel(model)
model = model.to(device)

# Определение оптимизатора и лосса
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = nn.SmoothL1Loss() #nn.MSELoss()

# Инициализация TensorBoard
writer = SummaryWriter()

# Процесс обучения с добавлением интерактивности
best_val_loss = float('inf')
patience = 3
no_improvement = 0

for epoch in range(3):
    model.train()
    train_loss = 0
    train_steps = 0
    with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}") as pbar:
        for batch in train_loader:
            input_ids1 = batch['input_ids1'].to(device)
            attention_mask1 = batch['attention_mask1'].to(device)
            input_ids2 = batch['input_ids2'].to(device)
            attention_mask2 = batch['attention_mask2'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids1, attention_mask1, input_ids2, attention_mask2)
            loss = loss_fn(outputs, labels)
            train_loss += loss.item()
            train_steps += 1
            loss.backward()
            optimizer.step()

            pbar.update(1)
            pbar.set_postfix({'loss': loss.item()})
            
            writer.add_scalar('Train/Loss', loss.item(), epoch * len(train_loader) + train_steps)

    avg_train_loss = train_loss / train_steps
    print(f"Average train loss for epoch {epoch}: {avg_train_loss}")

    model.eval()
    val_loss = 0
    val_steps = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids1 = batch['input_ids1'].to(device)
            attention_mask1 = batch['attention_mask1'].to(device)
            input_ids2 = batch['input_ids2'].to(device)
            attention_mask2 = batch['attention_mask2'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids1, attention_mask1, input_ids2, attention_mask2)
            loss = loss_fn(outputs, labels)
            val_loss += loss.item()
            val_steps += 1

            writer.add_scalar('Validation/Loss', loss.item(), epoch * len(val_loader) + val_steps)

    avg_val_loss = val_loss / val_steps
    print(f"Average validation loss for epoch {epoch}: {avg_val_loss}")

    # Логирование среднего значения потерь
    writer.add_scalar('Train/Average Loss', avg_train_loss, epoch)
    writer.add_scalar('Validation/Average Loss', avg_val_loss, epoch)

    # Проверка раннего прекращения
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        no_improvement = 0
        # Сохранение лучшей модели
        model.pretrained_model.save_pretrained('pretrained_model')
    else:
        no_improvement += 1
        if no_improvement >= patience:
            print(f"No improvement for {patience} consecutive epochs, stopping training.")
            
            break

writer.close()


Epoch 1: 100%|███████████| 23341/23341 [2:54:01<00:00,  2.24it/s, loss=0.000707]


Average train loss for epoch 0: 0.0024505393390420486
Average validation loss for epoch 0: 0.07701048382410877


Epoch 2: 100%|████████████| 23341/23341 [2:54:34<00:00,  2.23it/s, loss=0.00328]


Average train loss for epoch 1: 0.002092724632258371
Average validation loss for epoch 1: 0.06886259662477594


Epoch 3:  89%|██████████▋ | 20776/23341 [2:41:28<20:27,  2.09it/s, loss=0.00136]

In [ ]:
# Сохранение модели после предобучения
model.pretrained_model.save_pretrained('pretrained_model')


In [ ]:
# import torch
# from torch.utils.data import Dataset, DataLoader
# from transformers import AutoTokenizer, AutoModelForMaskedLM, AdamW
# from sklearn.model_selection import train_test_split
# from torch.utils.tensorboard import SummaryWriter
# import torch.nn as nn
# from tqdm import tqdm
# from torch.cuda.amp import GradScaler, autocast

# # Загрузка токенизатора и модели
# tokenizer = AutoTokenizer.from_pretrained('pretrained_model')
# model = AutoModelForMaskedLM.from_pretrained('pretrained_model')

# # Настройка устройства
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)

# class CosineSimilarityDataset(Dataset):
#     def __init__(self, pairs, tokenizer, max_len=256):
#         self.pairs = pairs
#         self.tokenizer = tokenizer
#         self.max_len = max_len

#     def __len__(self):
#         return len(self.pairs)

#     def __getitem__(self, idx):
#         sent1, sent2, label = self.pairs[idx]

#         encoding1 = self.tokenizer(
#             sent1,
#             max_length=self.max_len,
#             padding='max_length',
#             truncation=True,
#             return_tensors='pt'
#         )

#         encoding2 = self.tokenizer(
#             sent2,
#             max_length=self.max_len,
#             padding='max_length',
#             truncation=True,
#             return_tensors='pt'
#         )

#         return {
#             'input_ids1': encoding1['input_ids'].flatten(),
#             'attention_mask1': encoding1['attention_mask'].flatten(),
#             'input_ids2': encoding2['input_ids'].flatten(),
#             'attention_mask2': encoding2['attention_mask'].flatten(),
#             'labels': torch.tensor(label, dtype=torch.float)
#         }

# train_dataset = CosineSimilarityDataset(train_pairs, tokenizer)
# val_dataset = CosineSimilarityDataset(val_pairs, tokenizer)

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
# val_loader = DataLoader(val_dataset, batch_size=64, num_workers=4, pin_memory=True)

# class CosineSimilarityModel(nn.Module):
#     def __init__(self, pretrained_model):
#         super(CosineSimilarityModel, self).__init__()
#         self.pretrained_model = pretrained_model
#         self.pooling = nn.AdaptiveAvgPool1d(1)
#         self.cosine_similarity = nn.CosineSimilarity(dim=1)

#     def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2):
#         outputs1 = self.pretrained_model(input_ids1, attention_mask=attention_mask1)
#         outputs2 = self.pretrained_model(input_ids2, attention_mask=attention_mask2)
        
#         pooled_output1 = self.pooling(outputs1.last_hidden_state.transpose(1, 2)).squeeze(-1)
#         pooled_output2 = self.pooling(outputs2.last_hidden_state.transpose(1, 2)).squeeze(-1)
        
#         cosine_sim = self.cosine_similarity(pooled_output1, pooled_output2)
#         return cosine_sim

# model = CosineSimilarityModel(model)
# model = model.to(device)

# # Определение оптимизатора и лосса
# optimizer = AdamW(model.parameters(), lr=5e-5)
# loss_fn = nn.SmoothL1Loss() # nn.MSELoss()
# accumulation_steps = 4
# # Инициализация TensorBoard
# writer = SummaryWriter()

# # Использование автоматического смешивания точности
# scaler = GradScaler()

# # Процесс обучения с добавлением интерактивности
# best_val_loss = float('inf')
# patience = 3
# no_improvement = 0

# for epoch in range(3):
#     model.train()
#     train_loss = 0
#     train_steps = 0
#     with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}") as pbar:
#         for batch in train_loader:
#             input_ids1 = batch['input_ids1'].to(device, non_blocking=True)
#             attention_mask1 = batch['attention_mask1'].to(device, non_blocking=True)
#             input_ids2 = batch['input_ids2'].to(device, non_blocking=True)
#             attention_mask2 = batch['attention_mask2'].to(device, non_blocking=True)
#             labels = batch['labels'].to(device, non_blocking=True)

#             optimizer.zero_grad()

#             with autocast():
#                 outputs = model(input_ids1, attention_mask1, input_ids2, attention_mask2)
#                 loss = loss_fn(outputs, labels)
            
#             train_loss += loss.item()
#             train_steps += 1

            
#             scaler.scale(loss).backward()
#             scaler.step(optimizer)
#             scaler.update()

#             pbar.update(1)
#             pbar.set_postfix({'loss': loss.item()})
            
#             writer.add_scalar('Train/Loss', loss.item(), epoch * len(train_loader) + train_steps)

#     avg_train_loss = train_loss / train_steps
#     print(f"Average train loss for epoch {epoch}: {avg_train_loss}")

#     model.eval()
#     val_loss = 0
#     val_steps = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids1 = batch['input_ids1'].to(device, non_blocking=True)
#             attention_mask1 = batch['attention_mask1'].to(device, non_blocking=True)
#             input_ids2 = batch['input_ids2'].to(device, non_blocking=True)
#             attention_mask2 = batch['attention_mask2'].to(device, non_blocking=True)
#             labels = batch['labels'].to(device, non_blocking=True)

#             with autocast():
#                 outputs = model(input_ids1, attention_mask1, input_ids2, attention_mask2)
#                 loss = loss_fn(outputs, labels)
                
#             val_loss += loss.item()
#             val_steps += 1

#             writer.add_scalar('Validation/Loss', loss.item(), epoch * len(val_loader) + val_steps)

#     avg_val_loss = val_loss / val_steps
#     print(f"Average validation loss for epoch {epoch}: {avg_val_loss}")

#     # Логирование среднего значения потерь
#     writer.add_scalar('Train/Average Loss', avg_train_loss, epoch)
#     writer.add_scalar('Validation/Average Loss', avg_val_loss, epoch)

#     # Проверка раннего прекращения
#     if avg_val_loss < best_val_loss:
#         best_val_loss = avg_val_loss
#         no_improvement = 0
#         # Сохранение лучшей модели
#         model.pretrained_model.save_pretrained('best_pretrained_model')
#     else:
#         no_improvement += 1
#         if no_improvement >= patience:
#             print(f"No improvement for {patience} consecutive epochs, stopping training.")
#             break

# writer.close()



In [ ]:
# # Сохранение модели после предобучения
# model.pretrained_model.save_pretrained('pretrained_model')
